In [14]:
import matplotlib.pyplot as plt # visualização de dados
import yfinance as yf # dados financeiros
import pandas as pd # manipulação de dados

# Machine learning
from sklearn.metrics import(
    r2_score,
    mean_absolute_error,
    mean_absolute_percentage_error,
)

from neuralprophet import NeuralProphet

• Download dos dados(passo 1)

In [3]:
asset_code = "BTC-USD" # Bitcoin
start_date = "2015-01-01"
final_date = "2025-01-01"

# Dados históricos (DataFrame)
asset_data = yf.download(asset_code, start=start_date, end=final_date, multi_level_index=False)
 
# Visualizar dados
asset_data

WARNING - (py.warnings._showwarnmsg) - /var/folders/gj/xq3l6h9d6dx44zqvjxthjdsr0000gn/T/ipykernel_64999/1821033444.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  asset_data = yf.download(asset_code, start=start_date, end=final_date, multi_level_index=False)

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Date,,,,,
2015-01-01,314.248993,320.434998,314.002991,320.434998,8036550
2015-01-02,315.032013,315.838989,313.565002,314.079010,7860650
2015-01-03,281.082001,315.149994,281.082001,314.846008,33054400
2015-01-04,264.195007,287.230011,257.612000,281.145996,55629100
2015-01-05,274.473999,278.341003,265.084015,265.084015,43962800
...,...,...,...,...,...
2024-12-27,94164.859375,97294.843750,93310.742188,95704.976562,52419934565
2024-12-28,95163.929688,95525.898438,94014.289062,94160.187500,24107436185
2024-12-29,93530.226562,95174.875000,92881.789062,95174.054688,29635885267


• Formatação dos dados(passo 2)

In [4]:
# Verificando a coluna 'Date' para o formato datetime, essencial para análises temporais
asset_data.index


DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', length=3653, freq=None)

In [5]:
# Caso fosse necessário coonverter datas para objetos datetime
# asset_data.index = pd.to_datetime(asset_data.index)

# Formatação de dados para coincindir com formato esperado pelo Modelo NeuralProphet
# Filtrando a coluna 'Close' e aplicando reset_index para mover o índice de data para uma coluna normal

asset_data = asset_data[["Close"]].reset_index()

# Modelo NeuralProphet exige que a coluna com as datas seja nomeada de "ds" e valores "y"
asset_data.columns = ["ds", "y"]

asset_data

,ds,y
0,2015-01-01,314.248993
1,2015-01-02,315.032013
2,2015-01-03,281.082001
3,2015-01-04,264.195007
4,2015-01-05,274.473999
...,...,...
3648,2024-12-27,94164.859375
3649,2024-12-28,95163.929688
3650,2024-12-29,93530.226562
3651,2024-12-30,92643.210938


• Treinando modelo(passo 3)

In [6]:
# Criando modelo 
neural_model = NeuralProphet() # No NeuralProphet podemos settar alguns parâmetros como epoch, recomendado manter o padrão, pois ele calcula automaticamente o numero de interações

# Treinando modelo com dados selecionados
neural_model.fit(asset_data)




WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.973% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_au

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (58) is too small than the required number                     for the learning rate finder (239). The results might not be optimal.
WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 64. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

Finding best initial lr: 100%|██████████| 239/239 [00:01<00:00, 178.21it/s]


Training: |          | 0/? [00:20<?, ?it/s, v_num=3, train_loss=0.024, reg_loss=0.000, MAE=6.04e+3, RMSE=8.14e+3, Loss=0.0239, RegLoss=0.000] 


,train_loss,reg_loss,MAE,RMSE,Loss,RegLoss,epoch
0,0.528067,0.0,52553.898438,65020.632812,0.522829,0.0,0
1,0.315870,0.0,35687.828125,43530.445312,0.315519,0.0,1
2,0.174592,0.0,23122.556641,27756.404297,0.172499,0.0,2
3,0.092164,0.0,15107.819336,18099.363281,0.091741,0.0,3
4,0.059662,0.0,11114.454102,13833.654297,0.059796,0.0,4
...,...,...,...,...,...,...,...
75,0.024085,0.0,6062.784668,8213.397461,0.024117,0.0,75
76,0.024091,0.0,6025.381836,8111.797852,0.023792,0.0,76
77,0.024056,0.0,6030.349609,8110.775879,0.023808,0.0,77
78,0.024048,0.0,5988.707031,8092.171875,0.023732,0.0,78


In [7]:
# Criando dataframe com previsões futuras para o próximo 1 ano (365 dias)
future_data = neural_model.make_future_dataframe(asset_data, periods=365)

# Fazendo previsões para o futuro
future_predictions = neural_model.predict(future_data)

# Fazendo previsões para trás (Histórico a ser avaliado)
historical_predictions = neural_model.predict(asset_data)

WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.973% of the data.
WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is depreca

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 61.31it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.973% of the data.
WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is depreca

Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 100.07it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [8]:
historical_predictions


,ds,y,yhat1,trend,season_yearly,season_weekly
0,2015-01-01,314.248993,-99.331329,-929.299194,860.485535,-30.517555
1,2015-01-02,315.032013,-303.396362,-923.935059,769.679504,-149.140762
2,2015-01-03,281.082001,-300.127380,-918.570923,678.767151,-60.323559
3,2015-01-04,264.195007,-294.393585,-913.206787,588.130981,30.682306
4,2015-01-05,274.473999,-283.936859,-907.842773,498.512695,125.393272
...,...,...,...,...,...,...
3648,2024-12-27,94164.859375,68795.117188,67688.890625,1255.419434,-149.196594
3649,2024-12-28,95163.929688,68849.843750,67739.421875,1170.654297,-60.235027
3650,2024-12-29,93530.226562,68904.812500,67789.953125,1084.119141,30.745321
3651,2024-12-30,92643.210938,68961.523438,67840.492188,995.598694,125.435204


In [12]:
future_predictions

,ds,y,yhat1,trend,season_yearly,season_weekly
0,2025-01-01,None,68797.500000,67941.554688,815.328125,40.622787
1,2025-01-02,None,68685.679688,67992.093750,724.063538,-30.476469
2,2025-01-03,None,68526.960938,68042.625000,633.394226,-149.056396
3,2025-01-04,None,68576.046875,68093.156250,543.145447,-60.256386
4,2025-01-05,None,68628.507812,68143.695312,454.062286,30.747637
...,...,...,...,...,...,...
360,2025-12-27,None,87349.828125,86133.773438,1276.137695,-60.087852
361,2025-12-28,None,87407.101562,86184.304688,1192.184448,30.614521
362,2025-12-29,None,87466.203125,86234.843750,1105.959106,125.397934
363,2025-12-30,None,87346.046875,86285.375000,1017.820374,42.855442


In [17]:
# Plotando modelo

neural_model.plot_components(future_predictions)

WARNING - (py.warnings._showwarnmsg) - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neuralprophet/plot_forecast_plotly.py:283: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




ImportError: Please install ipywidgets>=7.0.0 to use the FigureWidget class

In [ ]:
import sys
print(sys.executable)


/usr/local/bin/python3
